In [1]:
#%matplotlib notebook
import sys
import os
import glob

# Standard modules used through the notebook 
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

from bundle_adjust import ba_timeseries
import pickle

# Display and interface settings (just for the notebook interface)
%matplotlib inline
%load_ext autoreload
%autoreload 2
np.set_printoptions(linewidth=150)

### Check std single date (in the overlapping areas) and std along time

In [31]:
scene_name = 'RB_ZAF_0001'
ba_method = 'ba_global_12D_9k'

'''
timeline_ids = ['s4_20190608T075318Z__s4_20190608T075352Z__s4_20190608T075428Z',
                's4_20190617T075138Z__s4_20190617T075212Z__s4_20190617T075248Z',
                's103_20190627T074821Z__s103_20190627T074854Z__s103_20190627T074931Z',
                's106_20190629T104731Z__s106_20190629T104805Z__s106_20190629T104841Z',
                's107_20190630T104349Z__s107_20190630T104424Z__s107_20190630T104459Z',]

'''
timeline_ids = ['s4_20190608T075318Z__s4_20190608T075352Z__s4_20190608T075428Z',
 's4_20190617T075138Z__s4_20190617T075212Z__s4_20190617T075248Z',
 's103_20190627T074821Z__s103_20190627T074854Z__s103_20190627T074931Z',
 's106_20190629T104731Z__s106_20190629T104805Z__s106_20190629T104841Z',
 's107_20190630T104349Z__s107_20190630T104424Z__s107_20190630T104459Z',
 's106_20190709T103954Z__s106_20190709T104028Z__s106_20190709T104103Z',
 's110_20190712T104057Z__s110_20190712T104131Z__s110_20190712T104207Z',
 's3_20190717T082601Z__s3_20190717T082634Z__s3_20190717T082709Z',
 's4_20190718T075551Z__s4_20190718T075625Z__s4_20190718T075701Z',
 's103_20190719T075406Z__s103_20190719T075441Z__s103_20190719T075516Z',
 's106_20190723T104750Z__s106_20190723T104823Z__s106_20190723T104859Z',
 's107_20190729T104103Z__s107_20190729T104137Z__s107_20190729T104213Z']


median_std_per_date = []
for t_id in timeline_ids:
    wdir = 'exp/{}/4D_ba_{}/warp/{}'.format(scene_name, ba_method, t_id)
    fnames = glob.glob('{}/**/*.tif'.format(wdir), recursive=True)

    stacked_warps = np.dstack([np.array(Image.open(fn)) for fn in fnames])
    
    counts_per_coord = np.sum(1*~np.isnan(stacked_warps), axis=2)
    overlapping_coords_mask = counts_per_coord >= 2
    
    std_current_date = np.nanstd(stacked_warps, axis=2)
    std_current_date[~overlapping_coords_mask] = np.nan
    
    median_std_per_date.append(np.nanmedian(std_current_date, axis=(0, 1)))
    
    out_fn = 'exp/{}/4D_ba_{}/4Dstats/std_per_date/{}.tif'.format(scene_name, ba_method, t_id)
    os.makedirs(os.path.dirname(out_fn), exist_ok=True)
    Image.fromarray(std_current_date).save(out_fn)
    
    #fig = plt.figure(figsize=(30,15))
    #plt.imshow(np.nanstd(stacked_warps, axis=2));

    
    
################
# std along time
################
dsms_dir = 'exp/{}/4D_ba_{}/dsms'.format(scene_name, ba_method)
stacked_dsms = np.dstack([np.array(Image.open(fn)) for fn in glob.glob(dsms_dir + '/*.tif')])
counts_per_coord = np.sum(1*~np.isnan(stacked_dsms), axis=2)
overlapping_coords_mask = counts_per_coord >= 2
std_along_time = np.nanstd(stacked_dsms, axis=2)
std_along_time[~overlapping_coords_mask] = np.nan

stats4D_dir = 'exp/{}/4D_ba_{}/4Dstats'.format(scene_name, ba_method)
os.makedirs(stats4D_dir, exist_ok=True)
Image.fromarray(std_along_time).save('{}/std_along_time.tif'.format(stats4D_dir))


print('bUndle adjustment method:', ba_method)

print('average median std per date: {:.2f}'.format(np.mean(median_std_per_date)))

print(median_std_per_date)

print('median std along time: {:.2f}'.format(np.nanmedian(std_along_time, axis=(0, 1))))

bUndle adjustment method: ba_global_12D_9k
average median std per date: 0.51
[0.24842024, 0.97477627, 0.597723, 0.45825958, 0.3293805, 0.42757988, 0.39956903, 0.57284033, 0.24034882, 0.88285255, 0.4516468, 0.584033]
median std along time: 1.03


### Check reprojection error

In [3]:
# load scene and timeline indices

scene_name = 'RB_ZAF_0001'

input_dir = '/home/carlo/dsms-timeseries-for-roger'
output_dir = 'exp'

scene = ba_timeseries.Scene(input_dir, output_dir, scene_name, 'v1', \
                            compute_aoi_masks=True, use_aoi_masks_to_equalize_crops=False)
#scene.display_aoi()

source_dsms_dir = '{}/{}_dsms'.format(scene.src_dir, scene_name)
example_dsm_fname = os.path.join(source_dsms_dir, \
                                 's103_20190627T074821Z__s103_20190627T074854Z__s103_20190627T074931Z.tiff')
scene.characterize_from_example_dsm(example_dsm_fname)
#scene.display_dsm_mask()

timeline_indices = [0, 3, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16]

timeline_indices = timeline_indices[:3]

scene.get_timeline_attributes(timeline_indices, ['datetime', 'n_images'])

#############################################################
Loading scene RB_ZAF_0001...

Found 85 different dates in the scene timeline

Total images in timeline: 560
Successfully loaded scene RB_ZAF_0001
#############################################################


Scene RB_ZAF_0001 characterized using the following example dsm:
/home/carlo/dsms-timeseries-for-roger/RB_ZAF_0001/RB_ZAF_0001_dsms/s103_20190627T074821Z__s103_20190627T074854Z__s103_20190627T074931Z.tiff

index  |  datetime             |  n_images
__________________________________________

0      |  2019-06-08 07:53:52  |  5       
3      |  2019-06-17 07:51:38  |  9       
6      |  2019-06-27 07:48:21  |  7       
7      |  2019-06-29 10:47:31  |  8       
8      |  2019-06-30 10:44:24  |  4       
9      |  2019-07-09 10:39:54  |  8       
11     |  2019-07-12 10:41:31  |  5       
12     |  2019-07-17 08:26:01  |  8       
13     |  2019-07-18 07:56:25  |  6       
14     |  2019-07-19 07:54:06  |  7       
15   

In [5]:
def eval_reprojection_error_from_rpcs(scene, myimages, ba_method, C, pairs_to_triangulate):
    
    
    # load filenames and rpcs
    rpcs_init_dir = '{}/RPC_init'.format(scene.dst_dir)
    rpcs_init = ba_timeseries.load_rpcs_from_dir(myimages, rpcs_init_dir, suffix='RPC', verbose=True)
    rpcs_adj_dir = '{}/{}/RPC_adj'.format(scene.dst_dir, ba_method)
    rpcs_adj = ba_timeseries.load_rpcs_from_dir(myimages, rpcs_adj_dir, suffix='RPC_adj', verbose=True)
    
    # load image crops
    mycrops = ba_timeseries.load_image_crops(myimages, verbose=True)
    
    # init ba
    cam_model = 'Perspective'
    ba_input_data = {}
    ba_input_data['input_dir'] = scene.dst_dir
    ba_input_data['output_dir'] = scene.dst_dir
    ba_input_data['n_new'] = len(myimages)
    ba_input_data['n_adj'] = 0
    ba_input_data['image_fnames'] = myimages
    ba_input_data['crops'] = mycrops
    ba_input_data['masks'] = None
    ba_input_data['rpcs'] = rpcs_init
    ba_input_data['cam_model'] = cam_model
    ba_input_data['aoi'] = scene.aoi_lonlat
    from bundle_adjust.ba_pipeline import BundleAdjustmentPipeline
    ba_pipeline = BundleAdjustmentPipeline(ba_input_data)
    
    # approximate rpcs as proj matrices
    from bundle_adjust.ba_core import approximate_rpcs_as_proj_matrices
    P_crop_ba = approximate_rpcs_as_proj_matrices(rpcs_adj, mycrops, scene.aoi_lonlat, cam_model)
    ba_pipeline.P_crop_ba = P_crop_ba
        
    # init pts 3D
    from bundle_adjust.ba_triangulation import initialize_3d_points
    pts_3d = initialize_3d_points(ba_pipeline.input_P, C, pairs_to_triangulate, cam_model)
    pts_3d_ba = initialize_3d_points(P_crop_ba, C, pairs_to_triangulate, cam_model)
        
    # compute reprojection error
    ba_pipeline.pts_3d = pts_3d
    ba_pipeline.pts_3d_ba = pts_3d_ba
    ba_pipeline.C = C
    avg_init, avg_ba = [], []
    for im_idx in range(len(myimages)):
        e_init, e_ba, _, _, _ = ba_pipeline.compute_reproj_err_per_image(im_idx)
        avg_init.append(np.mean(e_init))
        avg_ba.append(np.mean(e_ba))
        
        #ba_pipeline.analyse_reproj_err_particular_image(im_idx, plot=True)
        
        print('{:02}: avg init err {:.2f}, avg ba err {:.2f}'.format(im_idx, avg_init[-1], avg_ba[-1]))
        
    
    print('\nMethod: {} bundle adjustment'.format(ba_method))
    print('-----------------------------------------------------------------')
    print('\navg init err per image: {:.2f}'.format(np.mean(avg_init)))
    print('\navg ba err per image: {:.2f}'.format(np.mean(avg_ba)))

In [26]:
ba_method = 'ba_out-of-core_12D_9k' #'out-of-core'

pickle_in = open('{}/{}/matches.pickle'.format(scene.dst_dir, ba_method),'rb')
pairwise_matches = pickle.load(pickle_in)
pickle_in = open('{}/{}/pairs_triangulation.pickle'.format(scene.dst_dir, ba_method),'rb')
pairs_to_triangulate = pickle.load(pickle_in)
pickle_in = open('{}/{}/filenames.pickle'.format(scene.dst_dir, ba_method),'rb')
myimages = pickle.load(pickle_in)


#forbbiden_pairs = [(1,12), (12,16)]
#if len(forbbiden_pairs) >0:
#    pairs_to_triangulate = list(set(pairs_to_triangulate) - set(forbbiden_pairs))

#allowed_im_indices = [14, 15, 16, 17, 18, 19, 20]
#allowed_pairs = [(im_i, im_j) for im_i in allowed_im_indices for im_j in allowed_im_indices if im_i != im_j and im_i<im_j]

#allowed_pairs = [(1,12), (12,16)]
#pairs_to_triangulate = list(set(pairs_to_triangulate).intersection(set(allowed_pairs)))


features = []
for fn in myimages:
    f_id = os.path.splitext(os.path.basename(fn))[0]
    features.append(pickle.load(open('{}/{}/features/{}.pickle'.format(scene.dst_dir, ba_method, f_id),'rb')))
    
from feature_tracks.ft_utils import feature_tracks_from_pairwise_matches
C, _ = feature_tracks_from_pairwise_matches(features, pairwise_matches, pairs_to_triangulate)

print('Using {} feature tracks\n'.format(C.shape))

print('pairs to triangulate:', pairs_to_triangulate)

C.shape before baseline check (168, 62067)
C.shape after baseline check (168, 29968)
Using (168, 29968) feature tracks

pairs to triangulate: [(1, 3), (2, 3), (1, 4), (0, 4), (9, 12), (6, 8), (7, 11), (10, 11), (6, 9), (5, 10), (7, 8), (8, 12), (7, 10), (7, 12), (10, 12), (8, 13), (5, 11), (6, 13), (6, 12), (7, 9), (15, 16), (15, 17), (16, 20), (18, 20), (14, 20), (15, 19), (14, 19), (15, 20), (14, 18), (16, 19), (14, 16), (17, 19), (22, 23), (23, 28), (24, 26), (25, 27), (25, 26), (22, 25), (21, 27), (22, 26), (23, 26), (21, 26), (24, 27), (21, 25), (21, 24), (22, 28), (30, 31), (29, 32), (29, 31), (34, 35), (37, 40), (34, 36), (35, 39), (37, 39), (33, 40), (33, 39), (34, 38), (35, 38), (34, 39), (36, 39), (33, 37), (36, 40), (33, 36), (41, 44), (42, 44), (43, 45), (42, 45), (48, 53), (49, 51), (47, 49), (48, 52), (50, 52), (50, 53), (46, 53), (47, 50), (46, 52), (47, 51), (46, 50), (49, 53), (46, 48), (47, 53), (55, 59), (55, 57), (54, 59), (54, 58), (56, 59), (56, 58), (61, 63), (62

In [27]:
eval_reprojection_error_from_rpcs(scene, myimages, ba_method, C, pairs_to_triangulate)

Loading 84 image rpcs / 84

Loading 84 image rpcs / 84

Loading 84 image crops / 84

Approximating RPCs as Perspective projection matrices
84 projection matrices / 84 (0 err)
Done!

Approximating RPCs as Perspective projection matrices
84 projection matrices / 84 (0 err)
Done!

00: avg init err 0.94, avg ba err 0.46
01: avg init err 1.46, avg ba err 0.57
02: avg init err 4.50, avg ba err 0.50
03: avg init err 6.62, avg ba err 0.52
04: avg init err 0.52, avg ba err 0.38
05: avg init err 0.96, avg ba err 0.50
06: avg init err 1.37, avg ba err 0.78
07: avg init err 2.08, avg ba err 0.80
08: avg init err 1.15, avg ba err 0.39
09: avg init err 1.19, avg ba err 0.48
10: avg init err 0.80, avg ba err 0.32
11: avg init err 0.61, avg ba err 0.30
12: avg init err 1.25, avg ba err 0.56
13: avg init err 0.94, avg ba err 0.34
14: avg init err 1.22, avg ba err 0.41
15: avg init err 1.42, avg ba err 0.64
16: avg init err 1.05, avg ba err 0.46
17: avg init err 1.33, avg ba err 0.43
18: avg init err 0.

In [408]:
P1 = np.array([[ 4.85128086e-02, -8.83537890e-02,  1.40268335e-03,  3.66886552e+04],
               [ 3.97942353e-02,  2.32994575e-02,  8.98703082e-02,  1.70862603e+04],
               [-1.08059602e-07, -5.74392857e-08,  7.96691137e-08,  1.00000000e+00]])

P2 =  np.array([[ 4.85869516e-02, -8.82992204e-02,  1.35108588e-03,  3.60184995e+04],
                [ 3.95414376e-02,  2.31577511e-02,  9.00036459e-02,  1.90280930e+04],
                [-1.08160881e-07, -5.78021362e-08,  7.91655350e-08,  1.00000000e+00]])

from bundle_adjust import ba_utils

Pr = ba_utils.relative_extrinsic_matrix_between_two_proj_matrices(P2, P1, verbose=True)

np.allclose(P2, P1 @ Pr)

[R1 | t1] = [R2 | t2] @ [R21 | t21] ? True
Found a rotation of 0.24595336226186001 degrees between both cameras



False

In [410]:
from bundle_adjust import ba_core

k1, r1, t1, o1 = ba_core.decompose_perspective_camera(P1)

ext1 = np.hstack(( r1, t1[:, np.newaxis] ))

In [411]:
np.allclose( P1, k1 @ ext1)

True

In [412]:
np.allclose(P2, k2 @ ext1 @ Pr)

True